In [ ]:
!pip install requests

In [ ]:
# Create a sample config.json if it does not exist.
import json
import os

config_file = "config.json"
if not os.path.exists(config_file):
    sample_config = {
  "dexscreener_api": "https://api.dexscreener.com/latest/dex/tokens/",
  "database": "crypto_data.db",
  "check_interval": 300,
  "filters": {
    "pump_threshold": 50,
    "rug_threshold": -90,
    "tier1_mcap_threshold": 1000000000,
    "min_volume_24h": 10000,
    "min_market_cap": 50000,
    "min_liquidity_usd": 10000,
    "cex_volume_threshold": 10000000,
    "cex_mcap_threshold": 50000000,
    "max_volume_liquidity_ratio": 50,
    "min_price_change_for_volume": 5
  },
  "blacklists": {
    "coins": [
      "0x1234567890abcdef1234567890abcdef12345678",
      "0xabcdef1234567890abcdef1234567890abcdef12"
    ],
    "developers": [
      "0xdeadbeef1234567890abcdef1234567890abcdef",
      "0xfeedface1234567890abcdef1234567890abcdef"
    ]
  },
  "pocket_universe_api": "https://api.pocketuniverse.app/v1/check",
  "pocket_universe_key": "YOUR_API_KEY_HERE"
}
    with open(config_file, "w") as f:
        json.dump(sample_config, f, indent=2)
    print(f"Created a sample {config_file}. Please modify it with your settings.")

Created a sample config.json. Please modify it with your settings.


In [ ]:
import requests
import sqlite3
import time
import json
from datetime import datetime

# Load configuration
with open("config.json", "r") as f:
    CONFIG = json.load(f)

# Constants from config
DEXSCREENER_API = CONFIG["dexscreener_api"]
DATABASE = CONFIG["database"]
CHECK_INTERVAL = CONFIG["check_interval"]
FILTERS = CONFIG["filters"]
BLACKLISTS = CONFIG["blacklists"]
RUGCHECK_API = "https://api.rugcheck.xyz/check"  # Hypothetical endpoint

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS coins (
                    token_address TEXT PRIMARY KEY,
                    name TEXT,
                    price_usd REAL,
                    volume_24h REAL,
                    market_cap REAL,
                    liquidity_usd REAL,
                    price_change_24h REAL,
                    timestamp TEXT,
                    status TEXT,
                    creator_address TEXT,
                    volume_authenticity TEXT)''')
    conn.commit()
    conn.close()

# Fetch data from Dexscreener API
def fetch_token_data(token_address):
    url = f"{DEXSCREENER_API}{token_address}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data.get("pairs", [])
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {token_address}: {e}")
        return []

# Check RugCheck.xyz for token status
def rugcheck_verify(token_address):
    try:
        response = requests.get(f"{RUGCHECK_API}?token={token_address}")
        response.raise_for_status()
        result = response.json()
        # Assuming result has a "status" field with "Good", "Risky", etc.
        return result.get("status", "Unknown") == "Good"
    except requests.exceptions.RequestException as e:
        print(f"RugCheck API error for {token_address}: {e}")
        return False  # Default to false if API fails

# Check for bundled supply (e.g., >50% in one wallet)
def check_bundled_supply(pair):
    # Dexscreener doesn't directly provide holder distribution, so we approximate
    # using market cap and liquidity as proxies for concentration risk
    market_cap = float(pair.get("fdv", 0))
    liquidity = float(pair["liquidity"]["usd"])
    creator_address = pair.get("info", {}).get("creator", {}).get("address", "")

    # Heuristic: If liquidity is a tiny fraction of market cap, supply might be bundled
    if market_cap > 0 and (liquidity / market_cap) < 0.01:  # Less than 1% liquidity
        return True, creator_address
    return False, creator_address

# Update blacklist in config
def update_blacklist(token_address, creator_address):
    with open("config.json", "r+") as f:
        config = json.load(f)
        if token_address not in config["blacklists"]["coins"]:
            config["blacklists"]["coins"].append(token_address)
        if creator_address and creator_address not in config["blacklists"]["developers"]:
            config["blacklists"]["developers"].append(creator_address)
        f.seek(0)
        json.dump(config, f, indent=2)
        f.truncate()
    print(f"Blacklisted token {token_address} and dev {creator_address}")

# Check blacklists
def is_blacklisted(token_address, creator_address):
    return (token_address in BLACKLISTS["coins"] or
            (creator_address and creator_address in BLACKLISTS["developers"]))

# Verify volume authenticity (unchanged from previous)
def verify_volume_authenticity(pair):
    volume_24h = float(pair["volume"]["h24"])
    liquidity = float(pair["liquidity"]["usd"])
    price_change_24h = float(pair.get("priceChange", {}).get("h24", 0))
    volume_5m = float(pair["volume"]["m5"])
    ratio = volume_24h / liquidity if liquidity > 0 else float('inf')
    if ratio > FILTERS["max_volume_liquidity_ratio"]:
        return "suspicious_high_ratio"
    if volume_24h > FILTERS["min_volume_24h"] and abs(price_change_24h) < FILTERS["min_price_change_for_volume"]:
        return "suspicious_low_movement"
    if volume_5m / volume_24h > 0.5:
        return "suspicious_spike"
    return "authentic"

# Categorize coin
def categorize_coin(pair):
    token_address = pair["baseToken"]["address"]
    price = float(pair["priceUsd"])
    volume = float(pair["volume"]["h24"])
    market_cap = float(pair.get("fdv", 0))
    price_change = float(pair.get("priceChange", {}).get("h24", 0))
    liquidity = float(pair["liquidity"]["usd"])
    creator_address = pair.get("info", {}).get("creator", {}).get("address", "")

    # Apply basic filters
    if (volume < FILTERS["min_volume_24h"] or
        market_cap < FILTERS["min_market_cap"] or
        liquidity < FILTERS["min_liquidity_usd"]):
        return None

    # Check blacklists
    if is_blacklisted(token_address, creator_address):
        print(f"Skipping blacklisted coin/dev: {pair['baseToken']['name']} ({creator_address})")
        return None

    # RugCheck verification
    if not rugcheck_verify(token_address):
        print(f"Skipping {pair['baseToken']['name']} - Not marked 'Good' by RugCheck")
        return None

    # Check for bundled supply
    is_bundled, creator = check_bundled_supply(pair)
    if is_bundled:
        update_blacklist(token_address, creator)
        print(f"Skipping {pair['baseToken']['name']} - Bundled supply detected")
        return None

    # Verify volume authenticity
    authenticity = verify_volume_authenticity(pair)
    if authenticity.startswith("suspicious"):
        print(f"Skipping {pair['baseToken']['name']} - Suspicious volume: {authenticity}")
        return None

    # Categorize
    status = "stable"
    if price_change >= FILTERS["pump_threshold"]:
        status = "pumped"
    elif price_change <= FILTERS["rug_threshold"]:
        status = "rugged"
    elif market_cap >= FILTERS["tier1_mcap_threshold"]:
        status = "tier-1"
    elif (volume > FILTERS["cex_volume_threshold"] and
          market_cap > FILTERS["cex_mcap_threshold"]):
        status = "possible_cex_listing"

    return {
        "token_address": token_address,
        "name": pair["baseToken"]["name"],
        "price_usd": price,
        "volume_24h": volume,
        "market_cap": market_cap,
        "liquidity_usd": liquidity,
        "price_change_24h": price_change,
        "timestamp": datetime.now().isoformat(),
        "status": status,
        "creator_address": creator_address,
        "volume_authenticity": authenticity
    }

# Save data to database (unchanged)
def save_to_db(coin_data):
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''INSERT OR REPLACE INTO coins
                 (token_address, name, price_usd, volume_24h, market_cap,
                  liquidity_usd, price_change_24h, timestamp, status, creator_address, volume_authenticity)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
              (coin_data["token_address"], coin_data["name"], coin_data["price_usd"],
               coin_data["volume_24h"], coin_data["market_cap"], coin_data["liquidity_usd"],
               coin_data["price_change_24h"], coin_data["timestamp"], coin_data["status"],
               coin_data["creator_address"], coin_data["volume_authenticity"]))
    conn.commit()
    conn.close()

# Analyze historical data (unchanged)
def analyze_trends():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute("SELECT * FROM coins WHERE status != 'stable' ORDER BY timestamp DESC LIMIT 10")
    trends = c.fetchall()
    conn.close()
    print("\nRecent Notable Coins:")
    for trend in trends:
        print(f"{trend[1]} ({trend[0]}): Status={trend[8]}, Price=${trend[2]}, "
              f"Change={trend[6]}%, Market Cap=${trend[4]:,.0f}, Authenticity={trend[10]}, Time={trend[7]}")

# Main bot loop
def run_bot(token_addresses):
    init_db()
    while True:
        for token in token_addresses:
            pairs = fetch_token_data(token)
            if not pairs:
                continue
            pair = max(pairs, key=lambda x: float(x["volume"]["h24"]))
            coin_data = categorize_coin(pair)
            if coin_data:
                save_to_db(coin_data)
                print(f"Processed {coin_data['name']} - Status: {coin_data['status']} - Authenticity: {coin_data['volume_authenticity']}")
        analyze_trends()
        print(f"Waiting {CHECK_INTERVAL // 60} minutes before next check...")
        time.sleep(CHECK_INTERVAL)

if __name__ == "__main__":
    sample_tokens = [
        "0x2170Ed0880ac9A755fd29B2688956BD959F933F8",  # WETH on BSC
        "0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c"   # WBNB
    ]
    print("Starting Dexscreener Bot with RugCheck Integration...")
    run_bot(sample_tokens)

Starting Dexscreener Bot with RugCheck Integration...
RugCheck API error for 0x2170Ed0880ac9A755fd29B2688956BD959F933F8: 404 Client Error: Not Found for url: https://api.rugcheck.xyz/check?token=0x2170Ed0880ac9A755fd29B2688956BD959F933F8
Skipping Ethereum Token - Not marked 'Good' by RugCheck
RugCheck API error for 0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c: 404 Client Error: Not Found for url: https://api.rugcheck.xyz/check?token=0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c
Skipping Wrapped BNB - Not marked 'Good' by RugCheck

Recent Notable Coins:
Waiting 5 minutes before next check...
RugCheck API error for 0x2170Ed0880ac9A755fd29B2688956BD959F933F8: 404 Client Error: Not Found for url: https://api.rugcheck.xyz/check?token=0x2170Ed0880ac9A755fd29B2688956BD959F933F8
Skipping Ethereum Token - Not marked 'Good' by RugCheck
RugCheck API error for 0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c: 404 Client Error: Not Found for url: https://api.rugcheck.xyz/check?token=0xbb4CdB9CBd36B01bD1cBaEBF